## Introduction

I would like to use Foursquare to help decide which place in Bangkok, Thailand will be the best to move to if the person was originally from Melbourne, Australia. Both of the countries may seem very different but I would like to explore which places in Bangkok, Thailand have similar facilities as in Melbourne, Australia so that moving to Bangkok from Melbourne will not be too much of a difference. 
    
 

## Data

Factors that will be affecting the results are:
1. Places in Bangkok
2. Places in Melbourne City
3. Common venues in each places (Bangkok & Melbourne City)

The data source that we will be extracting/getting our data from are Foursquare and <a>https://www.geonames.org</a>

Data needed for this analysis:
1. Places and coordinates of those places in Bangkok
2. Places and coordinates of those places in Melbourne City
3. Top 100 places within 1km/1000m of each places using Foursquare(Bangkok & Melbourne City)

##### We will drop columns country and Admin1 which is the state since all of the country and state will be the same. We will also drop column Unnamed:0 since it is irrelevant.

In [2]:
bangkok_data=pd.read_csv('bangkok_data.csv')
melbourne_city_data=pd.read_csv('melbourne_city_data.csv')

In [3]:
bangkok_data.drop('Unnamed: 0',axis=1,inplace=True)
bangkok_data.head()

,Place,Code,Country,Admin1,Latitude,Longitude
0,Wattana,10110,Thailand,Bangkok,13.708,100.456
1,Yannawa,10120,Thailand,Bangkok,13.708,100.456
2,Thawi Wattana,10170,Thailand,Bangkok,13.767,100.450
3,Pom Prap Sattru Phai,10100,Thailand,Bangkok,13.733,100.500
4,Samphanthawong,10100,Thailand,Bangkok,13.733,100.500


In [4]:
bangkok_data.drop(['Country','Admin1'],axis=1,inplace=True)
bangkok_data.head()

,Place,Code,Latitude,Longitude
0,Wattana,10110,13.708,100.456
1,Yannawa,10120,13.708,100.456
2,Thawi Wattana,10170,13.767,100.450
3,Pom Prap Sattru Phai,10100,13.733,100.500
4,Samphanthawong,10100,13.733,100.500


In [5]:
melbourne_city_data.drop(['Country','Admin2','Unnamed: 0'],axis=1,inplace=True)
melbourne_city_data.head()

,Place,Code,Latitude,Longitude
0,Melbourne,3000,-37.813,144.961
1,East Melbourne,3002,-37.813,144.984
2,West Melbourne,3003,-37.809,144.947
3,Melbourne,3004,-37.842,144.976
4,Docklands,3008,-37.818,144.944


Since there is 2 Melbourne I did some research and found out that Postal Code 3004 is indeed also Melbourne however postal code 3004 is also linked to a place called St. Kilda Road Central. To avoid confusion of having 2 Melbournes, I will rename Melbourne with postal code 3004 to St. Kilda Road Central.

In [6]:
melbourne_city_data.at[3,'Place']='St. Kilda Road Central'
melbourne_city_data

,Place,Code,Latitude,Longitude
0,Melbourne,3000,-37.813,144.961
1,East Melbourne,3002,-37.813,144.984
2,West Melbourne,3003,-37.809,144.947
3,St. Kilda Road Central,3004,-37.842,144.976
4,Docklands,3008,-37.818,144.944
5,North Melbourne,3051,-37.799,144.949


In [7]:
melbourne_city_data.shape

(6, 4)

In [8]:
bangkok_data.shape

(44, 4)

### Let's visualize the neighbourhoods in Bangkok

##### Let's get Bangkok's coordinates

In [9]:
address = 'Bangkok, TH'

geolocator = Nominatim(user_agent="th_explorer")
location = geolocator.geocode(address)
latitude_bk = location.latitude
longitude_bk = location.longitude
print('The geograpical coordinate of Bangkok are {}, {}.'.format(latitude_bk, longitude_bk))

The geograpical coordinate of Bangkok are 13.7542529, 100.493087.


##### Let's visualize the neighbourhoods

In [10]:
map_bangkok=folium.Map(location=[latitude_bk,longitude_bk],zoom_start=9)

for lat,long,neigh in zip(bangkok_data['Latitude'], bangkok_data['Longitude'], bangkok_data['Place']):
    label=neigh
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangkok)
    
map_bangkok

### Let's visualize neighbourhoods in Melbourne City

##### Let's get Melbourne City coordinates

In [11]:
address = 'Melbourne City, AU'

geolocator = Nominatim(user_agent="au_explorer")
location = geolocator.geocode(address)
latitude_mel = location.latitude
longitude_mel = location.longitude
print('The geograpical coordinate of Melbourne City are {}, {}.'.format(latitude_mel, longitude_mel))

The geograpical coordinate of Melbourne City are -37.827558100000005, 144.97985722310904.


##### Lets visualize the neighbourhood in Melbourne City

In [12]:
map_melbourne=folium.Map(location=[latitude_mel,longitude_mel],zoom_start=11)

for lat,long,neigh in zip(melbourne_city_data['Latitude'], melbourne_city_data['Longitude'], melbourne_city_data['Place']):
    label=neigh
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_melbourne)
    
map_melbourne

### Let's use Foursquare to get the top 100 places within 1km of these cities

In [13]:
CLIENT_ID = '24XUTNADDQSXS4ESGZ3YG5GRFMCFZ1NPCPS5X4TRXBWP12UP'
CLIENT_SECRET = 'EFAO0VCYT0X42HXFFPF2ZDWWC3CA4YR0CAFZGUNFPVD0HOVC'
VERSION = '20180605'

In [14]:
import requests

#### Melbourne City

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
melbourne_city_venues = getNearbyVenues(names=melbourne_city_data['Place'],
                                   latitudes=melbourne_city_data['Latitude'],
                                   longitudes=melbourne_city_data['Longitude']
                                  )




Melbourne
East Melbourne
West Melbourne
St. Kilda Road Central
Docklands
North Melbourne


In [17]:
print(melbourne_city_venues.shape)
melbourne_city_venues.head()

(483, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Melbourne,-37.813,144.961,Tipo 00,-37.813527,144.961978,Italian Restaurant
1,Melbourne,-37.813,144.961,Shortstop Coffee & Donuts,-37.811240,144.961000,Donut Shop
2,Melbourne,-37.813,144.961,Brick Lane,-37.811341,144.959816,Café
3,Melbourne,-37.813,144.961,Brother Baba Budan,-37.813445,144.962137,Coffee Shop
4,Melbourne,-37.813,144.961,Kirk's Wine Bar,-37.813661,144.961351,Wine Bar


### Bangkok

In [18]:
bangkok_venues = getNearbyVenues(names=bangkok_data['Place'],
                                   latitudes=bangkok_data['Latitude'],
                                   longitudes=bangkok_data['Longitude']
                                  )

Wattana
Yannawa
Thawi Wattana
Pom Prap Sattru Phai
Samphanthawong
Bang Kholame
Thung Khru
Rat Burana
Bang Bon
Bang Khun Thian
Phasi Charoen
Nong Khaem
Bang Khae
Taling Chan
Phra Nakhon
Lak Si
Sai Mai
Bang Khen
Khanna Yao
Lat Phrao
Saphan Sung B39
Bung Kum
Pra Vet
Suang Luang
Phra Khanong
Dusit
Wangthong Lang
Huai Khwang
Pahtum Wan
Ding Daeng
Bang Rak
Khlong Samwa
Min Buri
Lat Krabang
Nong Chok
Bangkok Yai
Thon Buri
Khlong San
Bang Plat
Bangkok Noi
Bang Su
Bang Kapi
Bang Na
Phaya Thai


In [19]:
print(bangkok_venues.shape)
bangkok_venues.head()

(2417, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wattana,13.708,100.456,Starbucks (สตาร์บัคส์),13.709113,100.452060,Coffee Shop
1,Wattana,13.708,100.456,ก๋วยเตี๋ยวต้มยำตำลึง นายเอก,13.709675,100.453821,Noodle House
2,Wattana,13.708,100.456,7-Eleven (เซเว่น อีเลฟเว่น),13.710411,100.455035,Convenience Store
3,Wattana,13.708,100.456,sorry i'm hungry,13.710255,100.454076,Burger Joint
4,Wattana,13.708,100.456,TORA Yakiniku x Café,13.705987,100.447110,BBQ Joint


## Analysing each neighbourhood

### Melbourne City

In [20]:
melbourne_onehot= pd.get_dummies(melbourne_city_venues[['Venue Category']],prefix="",prefix_sep="")
melbourne_onehot['Neighbourhood']=melbourne_city_venues['Neighborhood']
fixed_columns=[melbourne_onehot.columns[-1]]+list(melbourne_onehot.columns[:-1])
melbourne_onehot=melbourne_onehot[fixed_columns]
melbourne_onehot.head()


,Neighbourhood,African Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Australian Restaurant,Auto Dealership,Bakery,Bar,Beer Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Stop,Café,Cambodian Restaurant,Candy Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Dessert Shop,Dim Sum Restaurant,Dog Run,Donut Shop,Dumpling Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food & Drink Shop,Football Stadium,French Restaurant,Fried Chicken Joint,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Himalayan Restaurant,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lounge,Malay Restaurant,Market,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Movie Theater,Museum,Nature Preserve,Night Market,Outlet Mall,Park,Performing Arts Venue,Persian Restaurant,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Restaurant,Road,Rock Club,Salad Place,Sandwich Place,Scandinavian Restaurant,School,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Soba Restaurant,Spanish Restaurant,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Melbourne,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Melbourne,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Melbourne,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Melbourne,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Melbourne,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [21]:
melbourne_grouped = melbourne_onehot.groupby('Neighbourhood').mean().reset_index()
melbourne_grouped

,Neighbourhood,African Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Australian Restaurant,Auto Dealership,Bakery,Bar,Beer Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Stop,Café,Cambodian Restaurant,Candy Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Dessert Shop,Dim Sum Restaurant,Dog Run,Donut Shop,Dumpling Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food & Drink Shop,Football Stadium,French Restaurant,Fried Chicken Joint,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Himalayan Restaurant,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lounge,Malay Restaurant,Market,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Movie Theater,Museum,Nature Preserve,Night Market,Outlet Mall,Park,Performing Arts Venue,Persian Restaurant,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Restaurant,Road,Rock Club,Salad Place,Sandwich Place,Scandinavian Restaurant,School,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Soba Restaurant,Spanish Restaurant,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Docklands,0.000000,0.000000,0.000000,0.00,0.01,0.01,0.010000,0.01,0.020000,0.030000,0.010000,0.00,0.00,0.000000,0.01,0.00,0.01,0.01,0.010000,0.000000,0.140000,0.01,0.00,0.01,0.00,0.01,0.000000,0.060000,0.00,0.00,0.000000,0.00,0.00,0.00,0.02,0.00,0.000000,0.000000,0.00,0.02,0.00,0.01,0.000000,0.00,0.010000,0.02,0.00,0.00,0.00,0.00,0.01,0.000000,0.020000,0.010000,0.00,0.00,0.000000,0.040000,0.00,0.01,0.010000,0.000000,0.000000,0.03,0.03,0.00,0.02,0.000000,0.00,0.00,0.00000,0.00,0.00,0.00,0.00,0.02,0.01,0.00,0.00,0.00,0.000000,0.00,0.01,0.030000,0.00,0.000000,0.010000,0.000000,0.000000,0.010000,0.010000,0.03,0.00,0.00,0.01,0.01,0.00,0.01,0.01,0.02,0.01,0.00,0.01,0.00,0.01,0.01,0.03,0.02,0.00,0.000000,0.01,0.00,0.020000,0.000000,0.01,0.01,0.00,0.01,0.00,0.02,0.01,0.00,0.00,0.00,0.00
1,East Melbourne,0.000000,0.000000,0.000000,0.01,0.00,0.01,0.040000,0.00,0.000000,0.060000,0.010000,0.00,0.00,0.010000,0.00,0.00,0.00,0.00,0.010000,0.000000,0.180000,0.00,0.00,0.01,0.00,0.01,0.040000,0.060000,0.00,0.00,0.000000,0.00,0.00,0.04,0.01,0.00,0.000000,0.000000,0.01,0.00,0.00,0.01,0.000000,0.01,0.000000,0.01,0.02,0.02,0.01,0.00,0.00,0.000000,0.010000,0.010000,0.00,0.00,0.000000,0.040000,0.01,0.00,0.010000,0.000000,0.000000,0.00,0.04,0.00,0.00,0.000000,0.00,0.00,0.00000,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.01,0.000000,0.00,0.00,0.020000,0.00,0.000000,0.020000,0.000000,0.010000,0.000000,0.020000,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.01,0.00,0.00,0.010000,0.01,0.00,0.050000,0.000000,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.03,0.01,0.01
2,Melbourne,0.000000,0.000000,0.010000,0.00,0.00,0.01,0.000000,0.00,0.000000,0.080000,0.010000,0.01,0.01,0.000000,0.00,0.02,0.00,0.00,0.020000,0.000000,0.070000,0.00,0.02,0.03,0.01,0.01,0.040000,0.070000,0.00,0.01,0.000000,0.02,0.01,0.00,0.03,0.02,0.000000,0.010000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.02,0.00,0.000000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.010000,0.03,0.02,0.01,0.01,0.050000,0.01,0.01,0.00000,0.01,0.00,0.00,0.01,0.00,0.01,0.01,0.02,0.00,0.000000,0.00,0.00,0.010000,0.01,0.000000,0.020000,0.000000,0.010000,0.000000,0.000000,0.00,0.01,0.00,0.00

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Getting top 10 venues of each neighbourhood

In [23]:
top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
melbourne_venues_sorted = pd.DataFrame(columns=columns)
melbourne_venues_sorted['Neighbourhood'] = melbourne_grouped['Neighbourhood']

for ind in np.arange(melbourne_grouped.shape[0]):
    melbourne_venues_sorted.iloc[ind, 1:] = return_most_common_venues(melbourne_grouped.iloc[ind, :], top_venues)

melbourne_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Docklands,Café,Coffee Shop,Hotel,Italian Restaurant,Bar,Restaurant,Steakhouse,Park,Japanese Restaurant,Supermarket
1,East Melbourne,Café,Coffee Shop,Bar,Thai Restaurant,Hotel,Cocktail Bar,Cricket Ground,Australian Restaurant,Japanese Restaurant,Wine Bar
2,Melbourne,Bar,Café,Coffee Shop,Korean Restaurant,Cocktail Bar,Shopping Mall,Chinese Restaurant,Dessert Shop,Italian Restaurant,Thai Restaurant
3,North Melbourne,Café,Thai Restaurant,Bar,Indonesian Restaurant,Pizza Place,Persian Restaurant,Korean Restaurant,Indie Theater,Hotel,Hostel
4,St. Kilda Road Central,Café,Australian Restaurant,Hotel,Light Rail Station,Park,Coffee Shop,French Restaurant,Portuguese Restaurant,Plaza,Pizza Place


### Bangkok

In [24]:
bangkok_onehot= pd.get_dummies(bangkok_venues[['Venue Category']],prefix="",prefix_sep="")
bangkok_onehot['Neighbourhood']=bangkok_venues['Neighborhood']
fixed_columns=[bangkok_onehot.columns[-1]]+list(bangkok_onehot.columns[:-1])
bangkok_onehot=bangkok_onehot[fixed_columns]
bangkok_onehot.head()

,Neighbourhood,Accessories Store,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Beer Bar,Beer Garden,Big Box Store,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Buddhist Temple,Buffet,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Canal,Cantonese Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,Comfort Food Restaurant,Comic Shop,Convenience Store,Coworking Space,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donburi Restaurant,Donut Shop,Duty-free Shop,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fishing Spot,Flea Market,Floating Market,Flower Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden Center,Gas Station,Gay Bar,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Government Building,Grocery Store,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Leather Goods Store,Library,Lounge,Market,Massage Studio,Metro Station,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Noodle House,Office,Other Nightlife,Outdoor Sculpture,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Pool,Print Shop,Pub,Public Bathroom,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Resort,Restaurant,Salad Place,Sandwich Place,Satay Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shopping Mall,Shopping Plaza,Shrine,Smoothie Shop,Snack Place,Soccer Field,Som Tum Restaurant,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toll Plaza,Tonkatsu Restaurant,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wings Joint,Yoga Studio,Yoshoku Restaurant
0,Wattana,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wattana,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wattana,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Wattana,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Wattana,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [25]:
bangkok_grouped=bangkok_onehot.groupby('Neighbourhood').mean().reset_index()
bangkok_grouped.head()

,Neighbourhood,Accessories Store,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Beer Bar,Beer Garden,Big Box Store,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Buddhist Temple,Buffet,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Canal,Cantonese Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,Comfort Food Restaurant,Comic Shop,Convenience Store,Coworking Space,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donburi Restaurant,Donut Shop,Duty-free Shop,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fishing Spot,Flea Market,Floating Market,Flower Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden Center,Gas Station,Gay Bar,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Government Building,Grocery Store,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Leather Goods Store,Library,Lounge,Market,Massage Studio,Metro Station,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Noodle House,Office,Other Nightlife,Outdoor Sculpture,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Pool,Print Shop,Pub,Public Bathroom,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Resort,Restaurant,Salad Place,Sandwich Place,Satay Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shopping Mall,Shopping Plaza,Shrine,Smoothie Shop,Snack Place,Soccer Field,Som Tum Restaurant,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toll Plaza,Tonkatsu Restaurant,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wings Joint,Yoga Studio,Yoshoku Restaurant
0,Bang Bon,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0625,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.062500,0.0,0.0,0.0625,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.062500,0.0,0.062500,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.125000,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0625,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062500,0.000000,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.0,0.312500,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0625,0.0,0.062500,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0
1,Bang Kapi,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.020000,0.0,0.010000,0.0000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.01,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0100,0.04,0.0,0.100000,0.0,0.0,0.0,0.01,0.020000,0.0,0.020000,0.030000,0.0,0.0,0.0,0.01,0.0,0.01,0.0,0.02,0.0,0.0,0.060000,0.0,0.03,0.0,0.000000,0.0,0.010000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.01,0.000000,0.000000,0.0,0.000000,0.020000,0.01,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.030000,0.030000,0.0,0.0000,0.0,0.01,0.060000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,

#### Getting top 10 venues of each neighbourhood

In [26]:
top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
bangkok_venues_sorted = pd.DataFrame(columns=columns)
bangkok_venues_sorted['Neighbourhood'] = bangkok_grouped['Neighbourhood']

for ind in np.arange(melbourne_grouped.shape[0]):
    bangkok_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bangkok_grouped.iloc[ind, :], top_venues)

bangkok_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bang Bon,Noodle House,Grocery Store,Intersection,Café,Som Tum Restaurant,Department Store,Snack Place,Convenience Store,Chinese Restaurant,Bar
1,Bang Kapi,Coffee Shop,Thai Restaurant,Japanese Restaurant,Noodle House,Fast Food Restaurant,Clothing Store,Dessert Shop,Hotpot Restaurant,Ice Cream Shop,Steakhouse
2,Bang Khae,Convenience Store,Market,Noodle House,Bookstore,Tennis Court,Fast Food Restaurant,Bus Station,Dessert Shop,Department Store,Asian Restaurant
3,Bang Khen,Convenience Store,Noodle House,Asian Restaurant,Bookstore,Ramen Restaurant,Ice Cream Shop,Hotpot Restaurant,Steakhouse,Vietnamese Restaurant,Shopping Mall
4,Bang Kholame,Noodle House,Coffee Shop,Thai Restaurant,Food Truck,Gym / Fitness Center,Convenience Store,Café,Shopping Mall,Burger Joint,Som Tum Restaurant


## Now lets combine the Melbourne City data with the Bangkok data to cluster them

In [27]:
combined_venues=pd.concat([melbourne_city_venues,bangkok_venues],ignore_index=True)

In [28]:
combined_venues.shape

(2900, 7)

In [29]:
combined_onehot= pd.get_dummies(combined_venues[['Venue Category']],prefix="",prefix_sep="")
combined_onehot['Neighbourhood']=combined_venues['Neighborhood']
fixed_columns=[combined_onehot.columns[-1]]+list(combined_onehot.columns[:-1])
combined_onehot=combined_onehot[fixed_columns]


In [30]:
combined_grouped = combined_onehot.groupby('Neighbourhood').mean().reset_index()
combined_grouped.head()

,Neighbourhood,Accessories Store,African Restaurant,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,BBQ Joint,Badminton Court,Bakery,Bar,Beer Bar,Beer Garden,Big Box Store,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Cambodian Restaurant,Canal,Candy Store,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,Comedy Club,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donburi Restaurant,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Government Building,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,Hobby Shop,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lounge,Malay Restaurant,Market,Massage Studio,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,Night Market,Noodle House,Office,Other Nightlife,Outdoor Sculpture,Outlet Mall,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Bathroom,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Rock Club,Salad Place,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shopping Mall,Shopping Plaza,Shrine,Skating Rink,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,Som Tum Restaurant,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toll Plaza,Tonkatsu Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Yoshoku Restaurant
0,Bang Bon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0625,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.062500,0.0,0.0,0.0,0.0,0.0625,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.062500,0.0,0.0,0.0,0.0,0.062500,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.125000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0625,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [31]:
top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
combined_sorted = pd.DataFrame(columns=columns)
combined_sorted['Neighbourhood'] = combined_grouped['Neighbourhood']

for ind in np.arange(combined_grouped.shape[0]):
    combined_sorted.iloc[ind, 1:] = return_most_common_venues(combined_grouped.iloc[ind, :], top_venues)

combined_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bang Bon,Noodle House,Grocery Store,Café,Bar,Chinese Restaurant,Snack Place,Convenience Store,Som Tum Restaurant,Department Store,Intersection
1,Bang Kapi,Coffee Shop,Thai Restaurant,Noodle House,Fast Food Restaurant,Japanese Restaurant,Clothing Store,Ramen Restaurant,Dessert Shop,Shabu-Shabu Restaurant,Flea Market
2,Bang Khae,Convenience Store,Department Store,Dessert Shop,Market,Bus Station,Tennis Court,Noodle House,Fast Food Restaurant,Asian Restaurant,Bookstore
3,Bang Khen,Convenience Store,Noodle House,Asian Restaurant,Ramen Restaurant,Bookstore,Hotpot Restaurant,Ice Cream Shop,Vietnamese Restaurant,Steakhouse,Stadium
4,Bang Kholame,Noodle House,Coffee Shop,Thai Restaurant,Gym / Fitness Center,Food Truck,Shopping Mall,Market,Supermarket,Gas Station,Café


## Clustering Data

In [77]:
# set number of clusters
kclusters = 5

combined_grouped_clustering = combined_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(combined_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 1, 1, 0, 0, 3, 0, 3, 0], dtype=int32)

In [79]:
combined_data=pd.concat([melbourne_city_data,bangkok_data],ignore_index=True)

In [80]:
# add clustering labels
combined_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

combined_merged = combined_data


combined_merged = combined_merged.join(combined_sorted.set_index('Neighbourhood'), on='Place')

combined_merged.head()

,Place,Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Melbourne,3000,-37.813,144.961,3.0,Bar,Café,Coffee Shop,Korean Restaurant,Cocktail Bar,Shopping Mall,Dessert Shop,Italian Restaurant,Chinese Restaurant,Thai Restaurant
1,East Melbourne,3002,-37.813,144.984,2.0,Café,Coffee Shop,Bar,Thai Restaurant,Australian Restaurant,Cricket Ground,Cocktail Bar,Hotel,Japanese Restaurant,Wine Bar
2,West Melbourne,3003,-37.809,144.947,2.0,Café,Korean Restaurant,Italian Restaurant,Bar,Pub,Indian Restaurant,Burger Joint,Pizza Place,Malay Restaurant,Hotel
3,St. Kilda Road Central,3004,-37.842,144.976,2.0,Café,Australian Restaurant,Hotel,Light Rail Station,French Restaurant,Coffee Shop,Park,Golf Course,Nature Preserve,Food & Drink Shop
4,Docklands,3008,-37.818,144.944,2.0,Café,Coffee Shop,Hotel,Restaurant,Steakhouse,Bar,Italian Restaurant,Japanese Restaurant,Park,Supermarket


In [78]:
combined_merged.drop('Cluster Labels',axis=1,inplace=True)
combined_sorted.drop('Cluster Labels',axis=1,inplace=True)

In [81]:
combined_merged.dropna(subset=['Cluster Labels'],axis=0,inplace=True)

In [82]:
print(combined_merged.shape)


(47, 15)


In [83]:
combined_merged[['Cluster Labels']]=combined_merged[['Cluster Labels']].astype('int32')

### Lets see the clusters in the map

#### Australia

In [84]:
# create map
map_clusters = folium.Map(location=[latitude_mel, longitude_mel], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(combined_merged['Latitude'], combined_merged['Longitude'], combined_merged['Place'], combined_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Bangkok

In [85]:
# create map
map_clusters = folium.Map(location=[latitude_bk, longitude_bk], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(combined_merged['Latitude'], combined_merged['Longitude'], combined_merged['Place'], combined_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analysing each cluster

### Cluster 1

In [86]:
combined_merged.loc[combined_merged['Cluster Labels'] == 0, combined_merged.columns[[0] + list(range(5, combined_merged.shape[1]))]]

,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Wattana,Noodle House,Coffee Shop,Thai Restaurant,Gym / Fitness Center,Food Truck,Shopping Mall,Market,Supermarket,Gas Station,Café
7,Yannawa,Noodle House,Coffee Shop,Thai Restaurant,Gym / Fitness Center,Food Truck,Shopping Mall,Market,Supermarket,Gas Station,Café
9,Pom Prap Sattru Phai,Noodle House,Convenience Store,Asian Restaurant,Seafood Restaurant,Chinese Restaurant,Thai Restaurant,Dessert Shop,Café,Coffee Shop,Som Tum Restaurant
10,Samphanthawong,Noodle House,Convenience Store,Asian Restaurant,Seafood Restaurant,Chinese Restaurant,Thai Restaurant,Dessert Shop,Café,Coffee Shop,Som Tum Restaurant
11,Bang Kholame,Noodle House,Coffee Shop,Thai Restaurant,Gym / Fitness Center,Food Truck,Shopping Mall,Market,Supermarket,Gas Station,Café
14,Bang Bon,Noodle House,Grocery Store,Café,Bar,Chinese Restaurant,Snack Place,Convenience Store,Som Tum Restaurant,Department Store,Intersection
15,Bang Khun Thian,Noodle House,Grocery Store,Café,Bar,Chinese Restaurant,Snack Place,Convenience Store,Som Tum Restaurant,Department Store,Intersection
17,Nong Khaem,Thai Restaurant,Noodle House,Flea Market,Convenience Store,Badminton Court,Toy / Game Store,Café,Steakhouse,Asian Restaurant,Playground
20,Phra Nakhon,Noodle House,Asian Restaurant,Convenience Store,Chinese Restaurant,Coffee Shop,Hotpot Restaurant,Burger Joint,Seafood Restaurant,Thai Restaurant,Sporting Goods Shop
31,Dusit,Noodle House,Thai Restaurant,Coffee Shop,Convenience Store,Asian Restaurant,Café,Som Tum Restaurant,Chinese Restaurant,Fast Food Restaurant,Market


#### Cluster 2

In [87]:
combined_merged.loc[combined_merged['Cluster Labels'] == 1, combined_merged.columns[[0] + list(range(5, combined_merged.shape[1]))]]

,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Bang Khae,Convenience Store,Department Store,Dessert Shop,Market,Bus Station,Tennis Court,Noodle House,Fast Food Restaurant,Asian Restaurant,Bookstore
22,Sai Mai,Convenience Store,Noodle House,Asian Restaurant,Ramen Restaurant,Bookstore,Hotpot Restaurant,Ice Cream Shop,Vietnamese Restaurant,Steakhouse,Stadium
23,Bang Khen,Convenience Store,Noodle House,Asian Restaurant,Ramen Restaurant,Bookstore,Hotpot Restaurant,Ice Cream Shop,Vietnamese Restaurant,Steakhouse,Stadium
28,Pra Vet,Food Truck,Convenience Store,Asian Restaurant,Flea Market,Noodle House,Coffee Shop,Som Tum Restaurant,Boat or Ferry,Pier,Food
29,Suang Luang,Food Truck,Convenience Store,Asian Restaurant,Flea Market,Noodle House,Coffee Shop,Som Tum Restaurant,Boat or Ferry,Pier,Food
30,Phra Khanong,Convenience Store,Noodle House,Coffee Shop,Hotel,Fast Food Restaurant,Train Station,Italian Restaurant,Thai Restaurant,Ice Cream Shop,Chinese Restaurant
40,Nong Chok,Fast Food Restaurant,Asian Restaurant,Food Truck,Supermarket,Flea Market,Thai Restaurant,Park,Shopping Mall,Gym / Fitness Center,Fried Chicken Joint
41,Bangkok Yai,Convenience Store,Noodle House,Food,Seafood Restaurant,Pharmacy,Market,Beer Bar,Food & Drink Shop,Food Court,Flower Shop


#### Cluster 3

In [88]:
combined_merged.loc[combined_merged['Cluster Labels'] == 2, combined_merged.columns[[0] + list(range(5, combined_merged.shape[1]))]]

,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Melbourne,Café,Coffee Shop,Bar,Thai Restaurant,Australian Restaurant,Cricket Ground,Cocktail Bar,Hotel,Japanese Restaurant,Wine Bar
2,West Melbourne,Café,Korean Restaurant,Italian Restaurant,Bar,Pub,Indian Restaurant,Burger Joint,Pizza Place,Malay Restaurant,Hotel
3,St. Kilda Road Central,Café,Australian Restaurant,Hotel,Light Rail Station,French Restaurant,Coffee Shop,Park,Golf Course,Nature Preserve,Food & Drink Shop
4,Docklands,Café,Coffee Shop,Hotel,Restaurant,Steakhouse,Bar,Italian Restaurant,Japanese Restaurant,Park,Supermarket
5,North Melbourne,Café,Thai Restaurant,Bar,Persian Restaurant,Burger Joint,Dog Run,Cocktail Bar,Coffee Shop,Taco Place,Gym


#### Cluster 4

In [89]:
combined_merged.loc[combined_merged['Cluster Labels'] == 3, combined_merged.columns[[0] + list(range(5, combined_merged.shape[1]))]]

,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Melbourne,Bar,Café,Coffee Shop,Korean Restaurant,Cocktail Bar,Shopping Mall,Dessert Shop,Italian Restaurant,Chinese Restaurant,Thai Restaurant
8,Thawi Wattana,Coffee Shop,Thai Restaurant,Noodle House,Japanese Restaurant,Seafood Restaurant,Convenience Store,Café,Dessert Shop,Shabu-Shabu Restaurant,Market
12,Thung Khru,Coffee Shop,Thai Restaurant,Convenience Store,Chinese Restaurant,Hotpot Restaurant,Noodle House,Seafood Restaurant,BBQ Joint,Pub,Fast Food Restaurant
13,Rat Burana,Coffee Shop,Thai Restaurant,Convenience Store,Chinese Restaurant,Hotpot Restaurant,Noodle House,Seafood Restaurant,BBQ Joint,Pub,Fast Food Restaurant
16,Phasi Charoen,Coffee Shop,Japanese Restaurant,BBQ Joint,Hotpot Restaurant,Fast Food Restaurant,Pub,Asian Restaurant,Pizza Place,Shopping Mall,Convenience Store
19,Taling Chan,Coffee Shop,Thai Restaurant,Noodle House,Japanese Restaurant,Seafood Restaurant,Convenience Store,Café,Dessert Shop,Shabu-Shabu Restaurant,Market
26,Saphan Sung B39,Coffee Shop,Thai Restaurant,Noodle House,Fast Food Restaurant,Japanese Restaurant,Clothing Store,Ramen Restaurant,Dessert Shop,Shabu-Shabu Restaurant,Flea Market
27,Bung Kum,Coffee Shop,Thai Restaurant,Noodle House,Fast Food Restaurant,Japanese Restaurant,Clothing Store,Ramen Restaurant,Dessert Shop,Shabu-Shabu Restaurant,Flea Market
35,Ding Daeng,Convenience Store,Noodle House,Café,Coffee Shop,Steakhouse,Thai Restaurant,Restaurant,Food Court,Hostel,Som Tum Restaurant
36,Bang Rak,Coffee Shop,Japanese Restaurant,Hotel,Ramen Restaurant,Thai Restaurant,Café,Gay Bar,BBQ Joint,Restaurant,Chinese Restaurant


#### Cluster 5

In [90]:
combined_merged.loc[combined_merged['Cluster Labels'] == 4, combined_merged.columns[[0] + list(range(5, combined_merged.shape[1]))]]

,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Khlong Samwa,Intersection,Thai Restaurant,Restaurant,Department Store,Coffee Shop,Bus Station,Noodle House,Park,Farm,Floating Market
38,Min Buri,Intersection,Thai Restaurant,Restaurant,Department Store,Coffee Shop,Bus Station,Noodle House,Park,Farm,Floating Market


We can see that <b>Cluster 3</b> consist of almost all the places in Melbourne City. <b>Cluster 4</b> contains various places in Bangkok, Thailand and also Melbourne, Melbourne City. <b>Cluster 1, 2 and 5</b> consist of all places in Bangkok, Thailand. 

#### Since there is only Melbourne in Melbourne City that are relatively similar to neighbourhoods/places in Bangkok. I will cluster the neighbourhoods/places in Melbourne City to see which other neighbourhoods/places are relatively similar to Melbourne, Melbourne City.

In [91]:
# set number of clusters
kclusters = 2

melbourne_city_clustering = melbourne_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_mel = KMeans(n_clusters=kclusters, random_state=0).fit(melbourne_city_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_mel.labels_[0:10] 

array([1, 1, 1, 0, 0, 1], dtype=int32)

In [92]:
melbourne_venues_sorted.insert(0, 'Cluster Labels', kmeans_mel.labels_)

melbourne_city_merged = melbourne_city_data


melbourne_city_merged = melbourne_city_merged.join(melbourne_venues_sorted.set_index('Neighbourhood'), on='Place')

melbourne_city_merged.head() 

,Place,Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Melbourne,3000,-37.813,144.961,1,Bar,Café,Coffee Shop,Korean Restaurant,Cocktail Bar,Shopping Mall,Chinese Restaurant,Dessert Shop,Italian Restaurant,Thai Restaurant
1,East Melbourne,3002,-37.813,144.984,1,Café,Coffee Shop,Bar,Thai Restaurant,Hotel,Cocktail Bar,Cricket Ground,Australian Restaurant,Japanese Restaurant,Wine Bar
2,West Melbourne,3003,-37.809,144.947,1,Café,Korean Restaurant,Italian Restaurant,Bar,Pub,Indian Restaurant,Burger Joint,Bakery,Middle Eastern Restaurant,Dessert Shop
3,St. Kilda Road Central,3004,-37.842,144.976,0,Café,Australian Restaurant,Hotel,Light Rail Station,Park,Coffee Shop,French Restaurant,Portuguese Restaurant,Plaza,Pizza Place
4,Docklands,3008,-37.818,144.944,1,Café,Coffee Shop,Hotel,Italian Restaurant,Bar,Restaurant,Steakhouse,Park,Japanese Restaurant,Supermarket


In [93]:
# create map
mel_map_clusters = folium.Map(location=[latitude_mel, longitude_mel], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(melbourne_city_merged['Latitude'], melbourne_city_merged['Longitude'], melbourne_city_merged['Place'], melbourne_city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(mel_map_clusters)
       
mel_map_clusters

When we run k-means algorithm with 2 clusters we can see that <b>Docklands, East Melbourne, West Melbourne</b> are also similar to<b> Melbourne</b>. Hence, we can say that these areas are also relatively similar to <b> Cluster 2</b> in Bangkok, Thailand.

### Now let's see these places in Melbourne City that are relatively similar to the places in Bangkok, Thailand.

#### Melbourne City

In [94]:
mel=melbourne_city_merged.loc[melbourne_city_merged['Cluster Labels'] == 1, melbourne_city_merged.columns[[0] + list(range(5, melbourne_city_merged.shape[1]))]]
print(mel.shape[0])
mel

4


,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Melbourne,Bar,Café,Coffee Shop,Korean Restaurant,Cocktail Bar,Shopping Mall,Chinese Restaurant,Dessert Shop,Italian Restaurant,Thai Restaurant
1,East Melbourne,Café,Coffee Shop,Bar,Thai Restaurant,Hotel,Cocktail Bar,Cricket Ground,Australian Restaurant,Japanese Restaurant,Wine Bar
2,West Melbourne,Café,Korean Restaurant,Italian Restaurant,Bar,Pub,Indian Restaurant,Burger Joint,Bakery,Middle Eastern Restaurant,Dessert Shop
4,Docklands,Café,Coffee Shop,Hotel,Italian Restaurant,Bar,Restaurant,Steakhouse,Park,Japanese Restaurant,Supermarket


#### Bangkok (Cluster 2)

In [95]:
bangkok=combined_merged.loc[combined_merged['Cluster Labels'] == 3, combined_merged.columns[[0] + list(range(5, combined_merged.shape[1]))]]

In [96]:
bangkok.drop(0,axis=0,inplace=True)

In [97]:
print(bangkok.shape[0])
bangkok

14


,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Thawi Wattana,Coffee Shop,Thai Restaurant,Noodle House,Japanese Restaurant,Seafood Restaurant,Convenience Store,Café,Dessert Shop,Shabu-Shabu Restaurant,Market
12,Thung Khru,Coffee Shop,Thai Restaurant,Convenience Store,Chinese Restaurant,Hotpot Restaurant,Noodle House,Seafood Restaurant,BBQ Joint,Pub,Fast Food Restaurant
13,Rat Burana,Coffee Shop,Thai Restaurant,Convenience Store,Chinese Restaurant,Hotpot Restaurant,Noodle House,Seafood Restaurant,BBQ Joint,Pub,Fast Food Restaurant
16,Phasi Charoen,Coffee Shop,Japanese Restaurant,BBQ Joint,Hotpot Restaurant,Fast Food Restaurant,Pub,Asian Restaurant,Pizza Place,Shopping Mall,Convenience Store
19,Taling Chan,Coffee Shop,Thai Restaurant,Noodle House,Japanese Restaurant,Seafood Restaurant,Convenience Store,Café,Dessert Shop,Shabu-Shabu Restaurant,Market
26,Saphan Sung B39,Coffee Shop,Thai Restaurant,Noodle House,Fast Food Restaurant,Japanese Restaurant,Clothing Store,Ramen Restaurant,Dessert Shop,Shabu-Shabu Restaurant,Flea Market
27,Bung Kum,Coffee Shop,Thai Restaurant,Noodle House,Fast Food Restaurant,Japanese Restaurant,Clothing Store,Ramen Restaurant,Dessert Shop,Shabu-Shabu Restaurant,Flea Market
35,Ding Daeng,Convenience Store,Noodle House,Café,Coffee Shop,Steakhouse,Thai Restaurant,Restaurant,Food Court,Hostel,Som Tum Restaurant
36,Bang Rak,Coffee Shop,Japanese Restaurant,Hotel,Ramen Restaurant,Thai Restaurant,Café,Gay Bar,BBQ Joint,Restaurant,Chinese Restaurant
39,Lat Krabang,Thai Restaurant,Convenience Store,Coffee Shop,Noodle House,Café,Hotel,Gym / Fitness Center,Bike Trail,Shabu-Shabu Restaurant,Soup Place


## Results

Our analysis shows that only some places in Melbourne City that are similar to places in Bangkok, Thailand. Most of the places in Melbourne City are quite dissimilar to the places in Bangkok, Thailand. In our analysis we focus on venues that are 1km within each of the places. By using the data we got using Foursquare and k-means, we grouped the venues of all the places in Bangkok and Melbourne City into 5 clusters to find the similar places. Most of the clusters have only places in Melbourne City or places in Bangkok but not both. Only 1 cluster contains a mixture of both. Only Melbourne of Melbourne City was similar to some places in Bangkok. There are 14 places in Bangkok that are similar to places in Melbourne, Melbourne City.

Since there is only 1 places (Melbourne) that are similar to places in Bangkok, we ran k-means on Melbourne City data to find out places that are relatively similar to Melbourne. As a result, we found 3 other places that are similar to Melbourne, Melbourne City. Hence, we can say that these places are relatively similar to the places in Bangkok that are similar to Melbourne, Melbourne City.

## Discussion

In the result, we can observe that only Melbourne, Melbourne City was the most similar to some places in Bangkok. This maybe because the top 10 venues in Melbourne, Melbourne City has the most Asian restaurants compared to the other places in Melbourne City. There are 14 places in Bangkok that are similar to Melbourne, Melbourne City. Hence, we can say that it would be better for people that are thinking to move from Melbourne, Melbourne City to Bangkok to move to these 14 places in Bangkok due to the similarity of venues between these places.

However, we also found 3 other places in Melbourne City that are relatively similar to Melbourne, Melbourne City. Even though it might not be too similar to where they are coming from from Melbourne City it is still relatively similar or similar enough for them to not feel too foreign when moving to these 14 places in Bangkok. 

## Conclusion

The purpose of this analysis is to discover similar places between Melbourne City, Australia and Bangkok, Thailand so that people that are moving from Melbourne City to Bangkok or vice versa can move to a place that is not too foreign to them in terms of the venues that is available within 1km of these places. By analyzing the venues using Foursquare and frequency of venue categories of places in Melbourne City and Bangkok we were able to get the top places in Melbourne City and Bangkok. By clustering these data we were able to get places in Melbourne City and Bangkok that are similar to each other in terms of the type of venues near to these places.

Due to lack of places in Melbourne City that are similar to places in Bangkok. We also analyzed places in Melbourne City that are similar to Melbourne, Melbourne City so that places in Melbourne City that are relatively similar to places in Bangkok can also be discovered.